In [1]:
import pandas as pd  #data maniplulation
import requests  #for sending HTTP requests
from bs4 import BeautifulSoup #for web scraping
import nltk #for natural language processing
import re #for string manipulation.
from nltk.sentiment.vader import SentimentIntensityAnalyzer  #for sentiment analysis
from nltk.tokenize import word_tokenize  #for tokenize
from nltk.corpus import stopwords     #for all stopwords         
from nltk.tokenize import RegexpTokenizer #tokenize a string or text based on a regular expression 
from nltk.stem import WordNetLemmatizer   #for lemmatize a word
from textblob import TextBlob #for processing textual data
import warnings 
warnings.filterwarnings('ignore')

In [2]:
def getdetails(url):
    page = requests.get(url, headers={"User-Agent": "XY"})
    soup = BeautifulSoup(page.content, "html.parser")
    article_text = soup.find('div', class_='td-post-content').get_text()
    #create a tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    #tokenize the text
    tokens = tokenizer.tokenize(article_text)
    #read in txt file containing stopwords
    stopwords_file = open("All_stopwords.txt", "r") #All_stopwords is a text file which made by me which have all the 5 stopwords provided from company
    #create a set of stopwords
    All_stopwords = set()
    #loop through the text file
    for line in stopwords_file:
        line = line.replace("\n", "")
        All_stopwords.add(line)
    #remove stopwords
    filtered_tokens = [w for w in tokens if not w in All_stopwords]
    #create a lemmatizer
    lemmatizer = WordNetLemmatizer()
    #lemmatize the tokens
    lemmatized_tokens = [lemmatizer.lemmatize(w) for w in filtered_tokens] 
    #calculate word count
    word_count = len(lemmatized_tokens)
    #calculate avg word length
    avg_word_length = 0
    for i in lemmatized_tokens:
        avg_word_length += len(i)
    avg_word_length = avg_word_length/word_count
    #calculate syllable count per word
    syllable_count_per_word = 0
    for i in lemmatized_tokens:
        syllable_count_per_word += len(i.split('-'))
    syllable_count_per_word = syllable_count_per_word/word_count
    #calculate personal pronouns
    personal_pronouns = 0
    for w in lemmatized_tokens:
        if w in ["I", "me", "you", "he", "him", "she", "her", "it", "we", "us", "they", "them"]:
            personal_pronouns += 1
    #create a function to calculate the positive and negative scores
    sid = SentimentIntensityAnalyzer()
    polarity_score = sid.polarity_scores(article_text)
    #calculate positive and negative score
    positive_score = polarity_score['pos']
    negative_score = polarity_score['neg']
    #sentiment analysis such as polarity and subjectivity
    text = TextBlob(article_text)
    polarity = text.sentiment.polarity
    subjective = text.sentiment.subjectivity
    #calculate complex word count
    complex_words = 0
    for w in lemmatized_tokens:
        syllable_count = len(w.split('-'))
        if syllable_count > 2:
            complex_words += 1
    #calculate average sentence length
    sentences = article_text.split('.')
    avg_sentence_length = 0
    for s in sentences:
        avg_sentence_length += len(s.split())
    avg_sentence_length = avg_sentence_length/len(sentences)
    #calculate the Gunning Fog Index
    fog_index = 0.4 * (avg_sentence_length +(100 * complex_words/word_count))
    #complex word percentage
    complex_words_percentage = (complex_words/word_count)*100
    #calculate the average number of words per sentence
    avg_words_per_sentence = word_count/len(sentences)
    
    return[positive_score,negative_score,polarity,subjective,avg_sentence_length,complex_words_percentage,fog_index,avg_words_per_sentence,complex_words,
            word_count, syllable_count_per_word, personal_pronouns, avg_word_length]
    

In [8]:
output = pd.read_excel("Output Data Structure.xlsx")

In [9]:
output

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38,https://insights.blackcoffer.com/what-if-the-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39,https://insights.blackcoffer.com/what-jobs-wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40,https://insights.blackcoffer.com/will-machine-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41,https://insights.blackcoffer.com/will-ai-repla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,147,https://insights.blackcoffer.com/the-future-of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,148,https://insights.blackcoffer.com/big-data-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,149,https://insights.blackcoffer.com/business-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
for i in output.index:
    try:
        result = getdetails(output["URL"][i])
        if len(result)==13:
            for jdx,j in enumerate(output.columns[2:]):
                output[j][i]=result[jdx]
    except Exception as e:
        print(e)
        pass

'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'


In [11]:
output

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,0.145,0.044,0.136936,0.463364,22.974359,0.0,9.189744,13.743590,0.0,1072.0,1.0,0.0,7.060634
1,38,https://insights.blackcoffer.com/what-if-the-c...,0.149,0.084,0.074693,0.433596,19.901408,0.0,7.960563,9.225352,0.0,655.0,1.0,0.0,6.422901
2,39,https://insights.blackcoffer.com/what-jobs-wil...,0.075,0.036,0.111470,0.483201,20.000000,0.0,8.000000,10.941176,0.0,930.0,1.0,0.0,7.096774
3,40,https://insights.blackcoffer.com/will-machine-...,0.145,0.044,0.142050,0.487387,19.891566,0.0,7.956627,9.156627,0.0,760.0,1.0,0.0,6.448684
4,41,https://insights.blackcoffer.com/will-ai-repla...,0.147,0.050,0.053178,0.509139,21.317073,0.0,8.526829,11.231707,0.0,921.0,1.0,0.0,6.490771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,0.129,0.051,0.035488,0.402814,16.962963,0.0,6.785185,9.907407,0.0,535.0,1.0,6.0,6.553271
110,147,https://insights.blackcoffer.com/the-future-of...,0.110,0.020,0.076504,0.391713,18.987952,0.0,7.595181,11.506024,0.0,955.0,1.0,0.0,6.102618
111,148,https://insights.blackcoffer.com/big-data-anal...,0.091,0.092,0.040319,0.417892,15.917808,0.0,6.367123,9.109589,0.0,665.0,1.0,0.0,6.493233
112,149,https://insights.blackcoffer.com/business-anal...,0.170,0.004,0.208748,0.546448,23.161290,0.0,9.264516,12.967742,0.0,402.0,1.0,0.0,7.457711


In [42]:
output.to_excel('final Output.xlsx', index=False)